In [ ]:
!pip install pydantic[email]

In [ ]:
pip install mcp_odoo/.

In [7]:
pip install nest_asyncio

Note: you may need to restart the kernel to use updated packages.


In [1]:
from src.data.data_retriever import DataRetriever
from src.data.odoo_connector import OdooConnection
import asyncio
from config.settings import settings
import nest_asyncio
import pandas as pd
import numpy as np
nest_asyncio.apply()

In [2]:
odoo_connection = OdooConnection()
asyncio.run(odoo_connection.connect())
data_retriever = DataRetriever(odoo_connection=odoo_connection)

Connected to Odoo as albert.gil@yourtechtribe.com (uid: 430)
Odoo server version: {'server_version': '16.0+e-20250313', 'server_version_info': [16, 0, 0, 'final', 0, 'e'], 'server_serie': '16.0', 'protocol_version': 1}


# Datos empresas

In [3]:
asyncio.run(data_retriever.get_all_companies())

[Company(id=1, name='Grupo Viko Digital Marketing, S.A.', currency_id=(1, 'EUR')),
 Company(id=3, name='Elogia Media S.L.', currency_id=(1, 'EUR')),
 Company(id=2, name='Ibrands Medios Interactivos SL', currency_id=(1, 'EUR')),
 Company(id=5, name='Kraz Data Solutions SL', currency_id=(1, 'EUR')),
 Company(id=6, name='Marketing4ecommerce Digital Content SL', currency_id=(1, 'EUR')),
 Company(id=13, name='Octoplus Digital Shelf Optimization SL', currency_id=(1, 'EUR')),
 Company(id=7, name='Tandem Trade Marketing SL', currency_id=(1, 'EUR')),
 Company(id=11, name='DigitalPla2021, S.L.', currency_id=(1, 'EUR')),
 Company(id=14, name='Ideas y Estrategia Digital SL', currency_id=(1, 'EUR')),
 Company(id=8, name='INICIATIVAS VIRTUALES DE MEXICO', currency_id=(33, 'MXN')),
 Company(id=12, name='IBRANDS MEDIOS INTERACTIVOS DE MEXICO', currency_id=(33, 'MXN')),
 Company(id=9, name='MITTUM MARKETING RELACIONAL', currency_id=(33, 'MXN'))]

# Datos facturas

In [3]:
company_id = 1
invoices = asyncio.run(data_retriever.get_all_outbound_invoices(company_id))
invoices_df = pd.DataFrame([i.to_dict() for i in invoices])

No estoy seguro de si esto puede llegar a pasar
No estoy seguro de si esto puede llegar a pasar
No estoy seguro de si esto puede llegar a pasar
No estoy seguro de si esto puede llegar a pasar
No estoy seguro de si esto puede llegar a pasar
No estoy seguro de si esto puede llegar a pasar
No estoy seguro de si esto puede llegar a pasar
No estoy seguro de si esto puede llegar a pasar
No estoy seguro de si esto puede llegar a pasar
No estoy seguro de si esto puede llegar a pasar
No estoy seguro de si esto puede llegar a pasar
No estoy seguro de si esto puede llegar a pasar
No estoy seguro de si esto puede llegar a pasar
No estoy seguro de si esto puede llegar a pasar
No estoy seguro de si esto puede llegar a pasar
No estoy seguro de si esto puede llegar a pasar
No estoy seguro de si esto puede llegar a pasar
No estoy seguro de si esto puede llegar a pasar
No estoy seguro de si esto puede llegar a pasar
No estoy seguro de si esto puede llegar a pasar
No estoy seguro de si esto puede llegar 

In [4]:
invoices_df.columns.values

array(['id', 'name', 'move_type', 'payment_state', 'company_id',
       'partner_id', 'currency_id', 'amount_total', 'amount_residual',
       'invoice_date', 'invoice_date_due', 'journal_id', 'payment_dates',
       'paid_late', 'days_overdue'], dtype=object)

In [5]:
invoices_df["payment_state"].value_counts()

payment_state
paid        80
not_paid    16
partial      3
reversed     1
Name: count, dtype: int64

In [6]:
invoices_df["paid_late"].value_counts()

paid_late
True     46
False    14
Name: count, dtype: int64

In [7]:
invoices_df["partner_id"].value_counts()

partner_id
(10, Elogia Media S.L.)                            13
(11527, The Tropicfeel S.L.)                       12
(9250, Equipzilla, S.L.)                            7
(13, Marketing4ecommerce Digital Content SL)        5
(16, MITTUM MARKETING RELACIONAL)                   5
(12322, Byfacility, S.L.)                           5
(10436, Octoplus Digital Shelf Optimization SL)     5
(8527, Fundació Pasqual Maragall)                   5
(15, INICIATIVAS VIRTUALES DE MEXICO)               3
(18, DigitalPla2021, S.L.)                          3
(12, Kraz Data Solutions SL)                        3
(13260, Ideas y Estrategia Digital SL)              3
(8307, Carglass, S.L.U.)                            3
(123, María Sieiro Alfonsin)                        2
(12867, Ufinet Latam, S.L.U.)                       2
(12548, Barkibu S.L.)                               2
(13105, Aldeas Infantiles SOS De España)            2
(14, Tandem Trade Marketing SL)                     2
(8571, Google Ire

In [15]:
invoices_df["currency_id"].value_counts()

currency_id
(1, EUR)    100
Name: count, dtype: int64